## Setup (Installs, Data, Models)

In [1]:
!pip install llama-index
!pip install llama-index-core==0.10.42
!pip install llama-index-embeddings-openai
!pip install llama-index-postprocessor-flag-embedding-reranker
!pip install git+https://github.com/FlagOpen/FlagEmbedding.git
!pip install llama-index-graph-stores-neo4j
!pip install llama-cloud-services

  Using cached llama_index_core-0.10.42-py3-none-any.whl.metadata (2.4 kB)
Using cached llama_index_core-0.10.42-py3-none-any.whl (15.4 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.14.12
    Uninstalling llama-index-core-0.14.12:
      Successfully uninstalled llama-index-core-0.14.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-cloud-services 0.6.54 requires llama-index-core>=0.12.0, but you have llama-index-core 0.10.42 which is incompatible.
llama-index-indices-managed-llama-cloud 0.9.4 requires llama-index-core<0.15,>=0.13.0, but you have llama-index-core 0.10.42 which is incompatible.
llama-index 0.14.12 requires llama-index-core<0.15.0,>=0.14.12, but you have llama-index-core 0.10.42 which is incompatible.
llama-index-readers-file 0.5.6 requires llama-index-core<0.15,>=0.13.0, but you have llama

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import os

# API access to llama-cloud
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-..."
os.environ["OPENAI_API_KEY"] = "sk-..."

#### Setup Model

Here we use gpt-4o and default OpenAI embeddings.

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

llm = OpenAI(model="gpt-4o")
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

Settings.llm = llm
Settings.embed_model = embed_model

Settings.chunk_size = 1024
Settings.chunk_overlap = 20

print(Settings.context_window)

128000


#### Load Data

Here we load the PDF and parse it with LlamaParse.

In [7]:
from llama_cloud_services import LlamaParse

docs = LlamaParse(result_type="text").load_data("../data/drehmoment.pdf")

Started parsing the file under job_id 94ecb8f6-11de-4e4e-90d7-f91f94ac372d


In [8]:
from copy import deepcopy
from llama_index.core.schema import TextNode, Document
from llama_index.core import VectorStoreIndex


def get_sub_docs(docs):
    """Split docs into pages, by separator."""
    sub_docs = []
    for doc in docs:
        doc_chunks = doc.text.split("\n---\n")
        for doc_chunk in doc_chunks:
            sub_doc = Document(
                text=doc_chunk,
                metadata=deepcopy(doc.metadata),
            )
            sub_docs.append(sub_doc)

    return sub_docs

In [9]:
# this will split into pages
sub_docs = get_sub_docs(docs)

#### Initialize Graph Store

Here we use Neo4j but you can also use our other integrations like Nebula (see an [example notebook](https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/property_graph/property_graph_advanced.ipynb)).

To launch Neo4j locally, first ensure you have docker installed. Then, you can launch the database with the following docker command

```bash
docker run \
    -p 7474:7474 -p 7687:7687 \
    -v $PWD/data:/data -v $PWD/plugins:/plugins \
    --name neo4j-apoc \
    -e NEO4J_apoc_export_file_enabled=true \
    -e NEO4J_apoc_import_file_enabled=true \
    -e NEO4J_apoc_import_file_use__neo4j__config=true \
    -e NEO4JLABS_PLUGINS=\[\"apoc\"\] \
    neo4j:latest
```

From here, you can open the db at [http://localhost:7474/](http://localhost:7474/). On this page, you will be asked to sign in. Use the default username/password of `neo4j` and `neo4j`.

Once you login for the first time, you will be asked to change the password.

After this, you are ready to create your first property graph!

In [10]:
from llama_index.graph_stores.neo4j import Neo4jPGStore

graph_store = Neo4jPGStore(
    username="neo4j",
    password="graph1312",
    url="bolt://localhost:7687",
)
vec_store = None

## Construct Knowledge Graph, Get Retrievers

This section shows you how to construct the knowledge graph over the existing documents.

**Note**: we have the default extractors (implicit path, simple llm path) configured. You can also choose to use a pre-defined schema as mentioned in this [notebook](https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/property_graph/property_graph_advanced.ipynb).

In [11]:
from llama_index.core.indices.property_graph import (
    ImplicitPathExtractor,
    SimpleLLMPathExtractor,
)
from llama_index.core import PropertyGraphIndex
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

In [12]:
index = PropertyGraphIndex.from_documents(
    sub_docs,
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
    kg_extractors=[
        ImplicitPathExtractor(),
        SimpleLLMPathExtractor(
            llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3),
            num_workers=4,
            max_paths_per_chunk=10,
        ),
    ],
    property_graph_store=graph_store,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.07s/it]


## Neo4j

To see the entire graph -> run cypher command "match n=() return n"
To delete the entire graph -> run cypher command "match n=() detach delete n"

In [11]:
# run this if index is already loaded
index = PropertyGraphIndex.from_existing(
    graph_store,
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
    kg_extractors=[
        ImplicitPathExtractor(),
        SimpleLLMPathExtractor(
            llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3),
            num_workers=4,
            max_paths_per_chunk=10,
        ),
    ],
    show_progress=True,
)

#### Define Vector Retriever

Here we define our vector context retriever - it returns initial nodes via vector search, and traverses the relations to pull in more nodes/context.

In [26]:
from llama_index.core.indices.property_graph import VectorContextRetriever

kg_retriever = VectorContextRetriever(
    index.property_graph_store,
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
    similarity_top_k=2,
    path_depth=1,
    # include_text=False,
    include_text=True,
)

In [27]:
nodes = kg_retriever.retrieve(
    # "Give me all the modules possible for the Spur gears made of polyacetal (POM)."
    "Give me the entire row of the bevel gear from PK with M=3,5"
)
# nodes = kg_retriever.retrieve('san francisco')
print(len(nodes))
for idx, node in enumerate(nodes):
    print(f">> IDX: {idx}, {node.get_content()}")

1
>> IDX: 0, Here are some facts extracted from the provided text:

Berechnung von kegelrädern -> Hat -> Zahnradabmessungen
Berechnung von kegelrädern -> Hat -> Zahnradabmessungen

Berechnung von Kegelrädern                                                      Kegelrad 1

1. Zahnradabmessungen für Kegelräder mit gerader Verzahnung                                              5

Teilkreisdurchmesser ØTK       ØTK =                  M·ZZ

Kopfkreisdurchmesser ØKK       ØKK = ØTK+2·M·COS δ

Kopfkegelwinkel Rad 1 y,      tany =           ZZ1+ 2 ·cosδ$                     Kegelrad 2              TK2
                                               ZZ2-2·cosδ1                                               KK2

Kopfkegelwinkel Rad 2 yz      tany2 =          ZZ2 + 2 ·coS δ2                  Abb. 1
                                               ZZ1-2·sinδ2

 Teilkegelwinkel Rad 1 δ,      tanδ}= ØTK1 = ZZ1 =1
                                      ØTK2            ZZ2         T

Teilkegelwinkel Rad 

## Build Baseline Vector Index

We also build a "baseline" vector index. This follows the "naive" RAG pipeline approach of chunking and vector embedding. We use this as a comparison point.

In [13]:
from llama_index.core import VectorStoreIndex
from llama_index.core.query_engine import RetrieverQueryEngine

base_index = VectorStoreIndex.from_documents(sub_docs, embed_model=embed_model)
base_retriever = base_index.as_retriever(similarity_top_k=10)
base_query_engine = RetrieverQueryEngine(base_retriever)

In [14]:
response = base_query_engine.query(
    "What is the document about? Answer in 2-3 sentences"
)
print(str(response))

The document provides technical specifications and guidelines for bevel gears made from Polyketon (PK) with a 1:1 gear ratio. It includes detailed measurements, torque information, and instructions for proper installation and customization options. Additionally, it addresses considerations for plastic gear tolerances and the importance of correct assembly to prevent noise and wear.


In [15]:
response = base_query_engine.query(
    "Based on the document, what are the suitable fit and tolerance for the bevel gear from Polyketon (PK)?"
)
print(str(response))

The suitable fit and tolerance for the bevel gear made from Polyketon (PK) are specified with reamed holes using H7/H8 tolerances. However, due to the plastic behavior of the material and temperature influences, the bore diameters can be narrowed by 0.02-0.04 mm. The product tolerance for bore diameters is given as H9 in the delivery state.


In [18]:
response = base_query_engine.query(
    "Is there any information about the torque of the bevel gear from Polyketon (PK)? If yes, please summarize it briefly."
)
print(str(response))

Yes, there is information about the torque of the bevel gear made from Polyketon (PK). The torque values provided in the tables refer exclusively to the gear teeth and do not consider factors like shaft diameter or key size. The calculations for load capacity are based on the pitting resistance of the tooth flanks and the tooth root stress. The torque values are specified per tooth, and the coverage, which is essential for determining the transmittable torque, depends on factors like the pitch circle diameter and gear pairing. In practice, a coverage degree of 1.1 to 1.25 is common for straight-toothed gears.


In [21]:
response = base_query_engine.query(
    "Is there a specific value of the torque mentioned in the document? if yes, list them and mention the source, if not, say: i don't know"
)
print(str(response))

Yes, specific values of torque are mentioned in the document. They are listed in the table under the column labeled "DM**" with values such as 1.06 Ncm, 9.92 Ncm, 27.71 Ncm, 33.24 Ncm, 84.44 Ncm, 162.3 Ncm, 262.21 Ncm, and 408.61 Ncm.


In [22]:
response = base_query_engine.query(
    "Give me the entire row of the bevel gear from PK with M=3,5"
)
print(str(response))

The entire row for the bevel gear from PK with M=3,5 is:

3,5   16  18 33,3 56 61,4 49,5 14,6 15,8 28 33,1 408,61 KPK3516-1:1PK


In [23]:
print(len(response.source_nodes))
for node in response.source_nodes:
    print("---")
    print(node.get_content())

2
---
zipperle
  Antriebstechnik

  Kegelräder aus Polyketon (PK)

  Übersetzung 1:1

  Ausführung: gespritzt, Eingriffswinkel 20°, Bohrung ab Modul 1,5
  spanabhebend bearbeitet.
  MaBänderung vorbehalten.                                        20

  E
       Q
   M

    B
    ØN
    ØTK
    0KK

 M Z      ØB ON ØTK ØKK        NL   ZB  Q [mm]    G DM**  Art.-Nr.

0,5 16 [mm] [mm] [mm] [mm] [mm] [mm] [mm] [mm]    [g] [Ncm]
          3 7     8   8,7 10,5 6    2   8,1     8,1 0,27 1,06  KPK0516-1:1PK
1,0   16  5 12    16 17,6 18,4 8 4,7 13,6 13,6 1,79      9,92 KPK1016-1:1PK
1,0   30  6 15       30 31 25,1 7,6 7   13,3 15,4 5,38   27,71 KPK1030-1:1PK
1,5   16  8  18,5 24 26,4 25,8 10       7 16,2 18,4 5,38 33,24 KPK1516-1:1PK
2,0   16  10 21,9 32  34,9 30,4 9.7 10  18,3 21,2 9,69   84,44 KPK2016-1:1PK
2,5   16  12 25,2 40  43,5 37   11,5 12,3 22,9 25,517,94 162,3 KPK2516-1:1PK
3,0   16  14 28,8 48  52,3 43   13,2 13,8 25,7 29,227,81262,21 KPK3016-1:1PK
3,5   16  18 33,3 56 61,4 49,5 14,6

## Build Custom Retriever

Build joint retriever that combines vector and KG search.

In [28]:
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore
from typing import List


class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both KG vector search and direct vector search."""

    def __init__(self, kg_retriever, vector_retriever):
        self._kg_retriever = kg_retriever
        self._vector_retriever = vector_retriever

    def _retrieve(self, query_bundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""
        kg_nodes = self._kg_retriever.retrieve(query_bundle)
        vector_nodes = self._vector_retriever.retrieve(query_bundle)

        unique_nodes = {n.node_id: n for n in kg_nodes}
        unique_nodes.update({n.node_id: n for n in vector_nodes})
        return list(unique_nodes.values())

In [29]:
custom_retriever = CustomRetriever(kg_retriever, base_retriever)

In [33]:
nodes = custom_retriever.retrieve(
    "Give me the entire row of the bevel gear from PK with M=3,5"
)
print(len(nodes))
for idx, node in enumerate(nodes):
    print(f">> IDX: {idx}, {node.get_content()}")

3
>> IDX: 0, Here are some facts extracted from the provided text:

Berechnung von kegelrädern -> Hat -> Zahnradabmessungen
Berechnung von kegelrädern -> Hat -> Zahnradabmessungen

Berechnung von Kegelrädern                                                      Kegelrad 1

1. Zahnradabmessungen für Kegelräder mit gerader Verzahnung                                              5

Teilkreisdurchmesser ØTK       ØTK =                  M·ZZ

Kopfkreisdurchmesser ØKK       ØKK = ØTK+2·M·COS δ

Kopfkegelwinkel Rad 1 y,      tany =           ZZ1+ 2 ·cosδ$                     Kegelrad 2              TK2
                                               ZZ2-2·cosδ1                                               KK2

Kopfkegelwinkel Rad 2 yz      tany2 =          ZZ2 + 2 ·coS δ2                  Abb. 1
                                               ZZ1-2·sinδ2

 Teilkegelwinkel Rad 1 δ,      tanδ}= ØTK1 = ZZ1 =1
                                      ØTK2            ZZ2         T

Teilkegelwinkel Rad 

## Build Agent

Now that we have the retriever, we can treat it as a RAG pipeline tool, and wrap it with an agent that can perform basic CoT reasoning and maintain conversation memory over time.

In [34]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import RetrieverQueryEngine

kg_query_engine = RetrieverQueryEngine(custom_retriever)
kg_query_tool = QueryEngineTool(
    query_engine=kg_query_engine,
    metadata=ToolMetadata(
        name="query_tool",
        description="Provides information about row table lookups",
    ),
)

In [35]:
# from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent.workflow import ReActAgent
from llama_index.core.workflow import Context

agent = ReActAgent(
    tools=[kg_query_tool],
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)

# context to hold this session/state
ctx = Context(agent)

## Try out Queries

Now that the agent is setup, let's try out some queries.

In [38]:
handler = agent.run("List me the entire row of the bevel gear from PK with M=3,5 exactly as it shows in the dimension table", ctx=ctx)

In [39]:
from llama_index.core.agent.workflow import ToolCallResult, AgentStream

async for ev in handler.stream_events():
    # if isinstance(ev, ToolCallResult):
    #     print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: query_tool
Action Input: {"input": "entire row of the bevel gear from PK with M=3.5 in dimension table"}
Observation: The entire row for the bevel gear from PK with M=3.5 in the dimension table is as follows: 

- Module (M): 3.5
- Number of Teeth (z): 16
- Bore Diameter (d): 18 mm
- Outer Diameter (da): 33.3 mm
- Pitch Diameter (dp): 56 mm
- Tip Diameter (de): 61.4 mm
- Length (l): 49.5 mm
- Width (b): 14.6 mm
- Weight: 408.61 g
- Article Number: KPK3516-1:1PK

Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The entire row for the bevel gear from PK with M=3.5 in the dimension table is as follows:

- Module (M): 3.5
- Number of Teeth (z): 16
- Bore Diameter (d): 18 mm
- Outer Diameter (da): 33.3 mm
- Pitch Diameter (dp): 56 mm
- Tip Diameter (de): 61.4 mm
- Length (l): 49.5 mm
- Width (b): 14.6 mm
- Weight: 408.61 g
- Ar

In [ ]:
handler = agent.run(
    "I want to buy a bevel gear from PK with that satisfies the three following technical requirements; 1) a gear ratio of 1:1, 2) a module between 1,0 and 2,5, 3) a torque between 28,7 Ncm and 85,1 Ncm. Can u suggest a suitable gear from the catalogue? if yes, please list me the ENTIRE row exactly as it appears in the dimension table, its fit and tolerance as well as its angle of engagement.", 
    ctx=ctx)

In [43]:
from llama_index.core.agent.workflow import ToolCallResult, AgentStream

async for ev in handler.stream_events():
    # if isinstance(ev, ToolCallResult):
    #     print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

Thought: The current language of the user is: English. I need to find a suitable bevel gear from PK that meets the specified technical requirements and provide the entire row exactly as it appears in the dimension table, along with fit, tolerance, and angle of engagement.
Action: query_tool
Action Input: {'input': 'bevel gear from PK with gear ratio 1:1, module between 1.0 and 2.5, torque between 28.7 Ncm and 85.1 Ncm, entire row, fit, tolerance, angle of engagement'}
Observation: The suitable bevel gear from PK that meets the specified criteria is the KPK2016-1:1PK. Here is the entire row as it appears in the dimension table:

- Module (M): 2.0
- Number of teeth (Z): 16
- Bore diameter (ØB): 10 mm
- Outer diameter (ON): 21.9 mm
- Pitch circle diameter (ØTK): 32 mm
- Outside circle diameter (ØKK): 34.9 mm
- Normal length (NL): 30.4 mm
- Width (ZB): 9.7 mm
- Weight (Q): 9.69 grams
- Torque (DM): 84.44 Ncm
- Article number: KPK2016-1:1PK

Fit and tolerance: Bore diameters are processed w

In [44]:
handler = agent.run(
    "I want to buy a bevel gear from PK with that satisfies the three following technical requirements; 1) a gear ratio of 1:1, 2) a module between 1,0 and 2,5, 3) a torque between 28,7 Ncm and 85,1 Ncm. Can u suggest a suitable gear from the catalogue? if yes, please list me the ENTIRE row exactly as it appears in the dimension table, its fit and tolerance as well as its angle of engagement. P.S: if there are more than one matching gear, return all possible rows", 
    ctx=ctx)

In [45]:
from llama_index.core.agent.workflow import ToolCallResult, AgentStream

async for ev in handler.stream_events():
    # if isinstance(ev, ToolCallResult):
    #     print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

Thought: The current language of the user is: English. I need to find all suitable bevel gears from PK that meet the specified technical requirements and provide the entire row exactly as it appears in the dimension table, along with fit, tolerance, and angle of engagement.
Action: query_tool
Action Input: {'input': 'all bevel gears from PK with gear ratio 1:1, module between 1.0 and 2.5, torque between 28.7 Ncm and 85.1 Ncm, entire row, fit, tolerance, angle of engagement'}
Observation: The suitable bevel gears from PK that meet the specified criteria are:

1. KPK1516-1:1PK
   - Module (M): 1.5
   - Number of teeth (Z): 16
   - Bore diameter (ØB): 8 mm
   - Outer diameter (ON): 18.5 mm
   - Pitch diameter (ØTK): 24 mm
   - Outer circle diameter (ØKK): 26.4 mm
   - Length (NL): 25.8 mm
   - Width (ZB): 10 mm
   - Weight (Q): 5.38 grams
   - Torque (DM): 33.24 Ncm
   - Angle of engagement: 20°
   - Fit and tolerance: Bore diameters are processed with a fit tolerance of H9.

2. KPK2016-1